In [20]:
from dotenv import load_dotenv
load_dotenv()

import os
import requests
import json
import planet
import os
import copy
import asyncio
import math
import geopandas as gpd

from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

from tqdm import tqdm

aoi = gpd.read_file('/home/christopher.x.ren/earth-index-ml/places/ra_aoi_indonesia.geojson')


PL_API_KEY = os.getenv('PL_API_KEY')
GCP_CREDENTIALS = os.getenv('B64_GCP_CREDENTIALS')
auth = planet.Auth.from_key(PL_API_KEY)

BASEMAP_API_URL = 'https://api.planet.com/basemaps/v1/mosaics'

session = requests.Session()
session.auth = (PL_API_KEY, "")

response = requests.get(BASEMAP_API_URL, auth=session.auth)
response

<Response [200]>

In [4]:

def handle_pagination(session, url, params, key='items'):
    """
    Handle paginated URLs by making multiple requests and yielding individual items.

    Parameters:
        session (requests.Session): The session object to make HTTP requests.
        url (str): The base URL for the paginated resource.
        params (dict): Query parameters to be sent with the request.
        key (str, optional): The key in the response JSON containing the list of items. Defaults to 'items'.

    Yields:
        dict: Individual items from the paginated resource.

    Raises:
        requests.HTTPError: If any HTTP errors occur during the requests.

    """
    while True:
        # Make a GET request to the specified URL with the given parameters
        response = session.get(url, params=params)

        # Raise an exception if the response has an HTTP error status code
        response.raise_for_status()

        # Parse the response body as JSON
        body = response.json()

        # Iterate over each item in the 'key' list of the response body and yield it
        for item in body[key]:
            yield item

        # Check if there is a next page link in the response body
        if '_next' in body['_links']:
            # Update the URL to the next page URL
            url = body['_links']['_next']
        else:
            # If there is no next page link, break the loop and stop pagination
            break


def polygon_search(mosaic_name, geometry):
    """Searches for quad ID's within a polygon geometry using the Planet Basemaps API.

    Parameters:
        mosaic_name (str): The name of the mosaic to search within.
        geometry (dict): The polygon geometry to search with.

    Yields:
        dict: The quad IDs found within the polygon geometry.

    Raises:
        requests.exceptions.HTTPError: If any HTTP error occurs during the API requests.

    """
    base_url = 'https://api.planet.com/basemaps/v1'

    # Configure retry logic for handling rate limiting (status code 429)
    retries = Retry(total=5, backoff_factor=0.2, status_forcelist=[429])
    session.mount('https://', HTTPAdapter(max_retries=retries))

    # Retrieve the mosaic ID from the mosaic name
    rv = session.get(f'{base_url}/mosaics', params={'name__is': mosaic_name})
    rv.raise_for_status()
    mosaic_id = rv.json()['mosaics'][0]['id']

    url = None
    while True:
        if url is None:
            # Initial request to search for quads within the mosaic
            url = f'{base_url}/mosaics/{mosaic_id}/quads/search'
            rv = session.post(url, json=geometry)
        else:
            # Request subsequent pages of quad search results
            rv = session.get(url)
        rv.raise_for_status()
        response = rv.json()

        # Yield item information for each result item
        for item in response['items']:
            yield item

        # Check if there are more pages of results
        if '_next' in response['_links']:
            url = response['_links']['_next']
        else:
            break

mosaic_name = "ps_monthly_sen2_normalized_analytic_8b_sr_subscription_2024_05_mosaic"
quad_ids = []
geometry = eval(aoi.to_json())['features'][0]['geometry']
# Search for quad IDs and add them to a list.
for quad in polygon_search(mosaic_name, geometry):
    quad_ids.append(quad['id'])

In [5]:
async def create_and_deliver_order(order_params, client):
    '''Create an order and wait for it to delivered

    Parameters:
        order_params: An order request
        client: An Order client object
    '''
    with planet.reporting.StateBar(state='creating') as reporter:
        # Place an order to the Orders API
        order = await client.create_order(order_params)
        reporter.update(state='created', order_id=order['id'])
        # Wait while the order is being completed
        await client.wait(order['id'], callback=reporter.update_state,
                          max_attempts=0)
        
async def batch_lists_and_place_orders(quad_ids, order_params):
    """
    Process quad IDs in batches and create orders.

    Parameters:
        quad_ids (list): A list of quad IDs to be processed in batches.
        order_params (dict): The order parameters dictionary that contains the details of the order.

    """

    # Calculate the number of batches
    num_batches = math.ceil(len(quad_ids) / 100)

    # Create batched quad IDs lists
    batched_quad_ids = [
        quad_ids[i:i + 100] for i in range(0, len(quad_ids), 100)
    ]

    # Duplicate the order_params dictionary for each batch
    all_order_params = [
        copy.deepcopy(order_params) for _ in range(num_batches)
    ]

    # Assign batched quad IDs to each order_params dictionary
    for i, params in enumerate(all_order_params):
        params['products'][0]['quad_ids'] = batched_quad_ids[i]

    async with planet.Session() as ps:
        # The Orders API client
        client = ps.client('orders')

        # Create the order and deliver it to GCP for each batch
        await asyncio.gather(*[
            create_and_deliver_order(params, client) 
            for params in all_order_params
        ])

In [29]:
# Test with single quad_id
import base64
test_order_params = {
    "name": "test basemap order with single quad_id",
    "source_type": "basemaps", 
    "products": [
        {
            "mosaic_name": mosaic_name,
            "quad_ids": [quad_ids[0]]  # Just use first quad_id for testing
        }
    ],
    # "tools": [
    #     {
    #         "bandmath": {
    #             "b1": "b1",
    #             "b2": "b2", 
    #             "b3": "b3",
    #             "b4": "b4",
    #             "b5": "(b4-b3)/(b4+b3)",
    #             "pixel_type": "32R"
    #         }
    #     }
    # ],
    "delivery": {
        "google_earth_engine": {
            "project": "earthindex",
            "collection": "planet/ps_monthly_sen2_normalized_analytic_8b_sr_subscription",
            "credentials": GCP_CREDENTIALS,
        }
    }
}

In [30]:

await batch_lists_and_place_orders([quad_ids[0]], test_order_params)

03:16 - order a8e26113-c42a-4112-bbdc-e4b0bb0e1fcc - state: success


In [27]:
import sys
sys.path.insert(0, '/home/christopher.x.ren/earth-index-ml/demeter/crop_mapping/src')

from gee import initialize_ee_with_credentials
initialize_ee_with_credentials()

import geemap
import ee

image = ee.Image.loadGeoTIFF("gs://demeter-labs/tea/imagery/ps_monthly_sen2_normalized_analytic_8b_sr/7ca583ca-ae69-4685-8ffe-7d448b30aa37/ps_monthly_sen2_normalized_analytic_8b_sr_subscription_2024_05_mosaic/1574-1053_quad_bandmath.tif")

# Create a map centered on Indonesia
Map = geemap.Map(center=[0, 120], zoom=5)
Map


Map(center=[0, 120], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

In [33]:
ic = ee.ImageCollection("projects/earthindex/assets/planet/ps_monthly_sen2_normalized_analytic_8b_sr_subscription")
im = ic.first()
im.getInfo()['bands']

[{'id': 'B1',
  'data_type': {'type': 'PixelType',
   'precision': 'int',
   'min': 0,
   'max': 65535},
  'dimensions': [4096, 4096],
  'crs': 'EPSG:3857',
  'crs_transform': [4.77731426716,
   0,
   10762333.581064306,
   0,
   -4.77731426716,
   587036.3771548793]},
 {'id': 'B2',
  'data_type': {'type': 'PixelType',
   'precision': 'int',
   'min': 0,
   'max': 65535},
  'dimensions': [4096, 4096],
  'crs': 'EPSG:3857',
  'crs_transform': [4.77731426716,
   0,
   10762333.581064306,
   0,
   -4.77731426716,
   587036.3771548793]},
 {'id': 'B3',
  'data_type': {'type': 'PixelType',
   'precision': 'int',
   'min': 0,
   'max': 65535},
  'dimensions': [4096, 4096],
  'crs': 'EPSG:3857',
  'crs_transform': [4.77731426716,
   0,
   10762333.581064306,
   0,
   -4.77731426716,
   587036.3771548793]},
 {'id': 'B4',
  'data_type': {'type': 'PixelType',
   'precision': 'int',
   'min': 0,
   'max': 65535},
  'dimensions': [4096, 4096],
  'crs': 'EPSG:3857',
  'crs_transform': [4.77731426716

In [36]:
Map = geemap.Map(center=[0, 120], zoom=5)

Map.addLayer(im.select(['B6', 'B4', 'B2']),
             {'min': 0, 'max': 2500}, 'image')
Map


Map(center=[0, 120], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

In [1]:
import geopandas as gpd
from pathlib import Path
base_dir = '/home/christopher.x.ren/embeddings/ra_tea/planet_embeddings'
test_file = gpd.read_parquet(Path(base_dir) /'non_deduped'/'geo'/ 'ps_monthly_sen2_normalized_analytic_8b_sr_subscription_2024_02_mosaic_non_deduped_duckdb-geo.parquet')
test_file

ValueError: Missing geo metadata in Parquet/Feather file.
            Use pandas.read_parquet/read_feather() instead.